Download the dataset from https://www.kaggle.com/tongpython/cat-and-dog

## 1. Import library

In [1]:
import keras
import matplotlib.pyplot as plt
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image

## 2. Model create

In [2]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="softmax"))

## 3. Model compile

In [3]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

## 4. Image augmentation

In [5]:
##create an object of ImageDataGenerator, for augmenting train set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=15, 
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range = 0.1,
                                   vertical_flip=False,
                                   horizontal_flip = True,
                                   fill_mode="reflect")

##create another object of ImageDataGenerator, for augmenting test set
test_datagen = ImageDataGenerator(rescale = 1./255)

##apply image augmentation on train set by resizing all images to 64x64 and creating batches of 32 images.
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (244,224),
                                                 batch_size = 8)

##apply image augmentation on test set by resizing all images to 64x64 and creating batches of 32 images.
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (224,244),
                                            batch_size = 8)

Found 20 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


## 5. Model fit

In [6]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
#ModelCheckpoint callback saves a model at some interval. 
filepath="saved_models/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.h5" #File name includes epoch and validation accuracy.
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto'
                            )
#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto')

#Changing learning rate
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', patience=3, factor=0.01, verbose=1, min_lr=1e-5)

#CSVLogger logs epoch, acc, loss, val_acc, val_loss
log_csv = CSVLogger('my_logs.csv', separator=',', append=False)

In [7]:
###steps_per_epoch: num of data divided by batch size
###validation_steps: num of data divided by batch size
history = model.fit_generator(generator=training_set,
                         steps_per_epoch = (20/8),
                         epochs = 20,
                         validation_data = test_set,
                         validation_steps = (20/8),callbacks=[checkpoint,early,lr_reduce,log_csv])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
3/2 [====================================] - ETA: -1s - loss: 1.6453 - accuracy: 0.5000
Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to saved_models\weights-improvement-01-0.50.h5
3/2 [====================================] - 29s 10s/step - loss: 1.6453 - accuracy: 0.5000 - val_loss: 0.7037 - val_accuracy: 0.5000
Epoch 2/20
3/2 [====================================] - ETA: 0s - loss: 0.7171 - accuracy: 0.5000
Epoch 00002: val_accuracy did not improve from 0.50000
3/2 [====================================] - 6s 2s/step - loss: 0.7171 - accuracy: 0.5000 - val_loss: 4.7517 - val_accuracy: 0.5000
Epoch 3/20
3/2 [====================================] - ETA: 0s - loss: 1.0369 - accuracy: 0.5000
Epoch 00003: val_accuracy did not improve from 0.50000
3/2 [====================================] - 7s 2s/step - loss: 1.0369 - accuracy: 0.5000 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoc

In [8]:
model.save("vgg16_2.h5")